---
title: "Advanced Fault Classification"
format: html
---

# 🤖 Advanced Fault Classification & Predictive Maintenance
## Portfolio Project 5 — Deep Feature Engineering, Class Imbalance, Explainable AI (SHAP), Calibrated Probabilities & Remaining Useful Life Regression

---

### What This Notebook Covers (Beyond Basics)
| Topic | Technique |
|---|---|
| Deep feature engineering | FFT-based spectral features, autocorrelation, Hilbert envelope |
| Class imbalance | SMOTE (synthetic oversampling), class-weighted loss, threshold tuning |
| Explainability | SHAP TreeExplainer — global + local feature importance |
| Calibration | Platt scaling + reliability diagrams |
| RUL regression | Remaining Useful Life prediction via survival-style regression |
| Ensemble + stacking | Voting classifier with calibrated probability outputs |

### Dataset  
**NASA C-MAPSS Turbofan Engine** (synthetic replica, 4 fault classes + RUL)  
Reference: https://data.nasa.gov/Machinery-and-Dynamics/

---


In [ ]:
# ─── 1. Imports ─────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              VotingClassifier)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import (train_test_split, StratifiedKFold,
                                     cross_val_score)
from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_curve, auc, precision_recall_curve)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
print('✓ All imports loaded.')

In [ ]:
# ─── 2. Synthetic vibration + thermal data (C-MAPSS style) ─
def gen_cmapss_data(n_units=100, seed=2024):
    """
    Simulate n_units turbofan engines, each with a sequence of cycles.
    Each unit has a true RUL that decreases each cycle until failure.
    At each cycle we record 8 sensor readings.
    Classes: Normal, Fault_A (bearing), Fault_B (blade), Fault_C (seal).
    """
    rng = np.random.default_rng(seed)
    rows = []

    # Fault signatures (mean shift per sensor for each fault)
    fault_shifts = {
        'Normal':  np.zeros(8),
        'Fault_A': np.array([0.5, 0.3, 1.2, 0.1, 0.8, 0.0, 0.4, 0.2]),
        'Fault_B': np.array([0.1, 1.0, 0.2, 0.9, 0.1, 1.1, 0.3, 0.7]),
        'Fault_C': np.array([0.8, 0.2, 0.4, 0.3, 1.5, 0.2, 1.0, 0.6]),
    }
    # Assign each unit a failure type (30% normal lifetime, 70% develop faults)
    unit_types = rng.choice(['Normal', 'Fault_A', 'Fault_B', 'Fault_C'],
                            n_units, p=[0.30, 0.25, 0.25, 0.20])

    for unit in range(n_units):
        max_rul = rng.integers(80, 150)   # max useful life
        fault_type = unit_types[unit]
        # Fault onset: last 20-40% of life
        onset = int(max_rul * rng.uniform(0.60, 0.80))

        for cycle in range(max_rul):
            rul = max_rul - cycle - 1

            # Healthy baseline signal
            base = rng.normal(0, 0.15, 8)

            # Degradation ramp after onset
            if cycle > onset:
                deg_ratio = (cycle - onset) / (max_rul - onset)
                shift = fault_shifts[fault_type] * deg_ratio * 2.5
                base += shift
                label = fault_type
            else:
                label = 'Normal'

            rows.append({
                'Unit': unit, 'Cycle': cycle, 'RUL': rul,
                'Label': label,
                **{f'Sensor_{i}': round(base[i], 4) for i in range(8)}
            })

    return pd.DataFrame(rows)


df = gen_cmapss_data(n_units=120)
print(f'Shape: {df.shape}')
print('\nLabel distribution:')
print(df['Label'].value_counts())

In [ ]:
# 1. Deep Feature Engineering — Spectral + Temporal + Statistical

# ─── 3. Per-unit sliding-window feature extraction ────────
SENSOR_COLS = [f'Sensor_{i}' for i in range(8)]
WIN = 30   # sliding window size (cycles)


def extract_features(group, win=WIN):
    """Extract rich features from a sliding window of sensor data."""
    feats = []
    labels = []
    ruls = []
    for end in range(win, len(group)):
        chunk = group[SENSOR_COLS].iloc[end-win:end].values  # (win, 8)
        row = {}

        for i, col in enumerate(SENSOR_COLS):
            s = chunk[:, i]
            # --- Statistical ---
            row[f'{col}_mean'] = np.mean(s)
            row[f'{col}_std'] = np.std(s)
            row[f'{col}_skew'] = float(pd.Series(s).skew())
            row[f'{col}_kurt'] = float(pd.Series(s).kurtosis())
            row[f'{col}_ptp'] = np.ptp(s)
            row[f'{col}_rms'] = np.sqrt(np.mean(s**2))
            row[f'{col}_crest'] = np.max(
                np.abs(s)) / (np.sqrt(np.mean(s**2)) + 1e-8)

            # --- Trend (linear slope) ---
            x_idx = np.arange(len(s))
            row[f'{col}_slope'] = np.polyfit(x_idx, s, 1)[0]

            # --- FFT: dominant frequency energy ratio ---
            fft_vals = np.abs(np.fft.rfft(s - s.mean()))
            fft_energy = fft_vals**2
            total_e = fft_energy.sum() + 1e-10
            row[f'{col}_fft_dom_ratio'] = fft_energy[1:].max() / total_e
            row[f'{col}_fft_spectral_entropy'] = -np.sum(
                (fft_energy[1:]/total_e) * np.log2(fft_energy[1:]/total_e + 1e-10))

            # --- Autocorrelation at lag 1 ---
            if np.std(s) > 1e-8:
                row[f'{col}_acf1'] = np.corrcoef(s[:-1], s[1:])[0, 1]
            else:
                row[f'{col}_acf1'] = 0.0

        feats.append(row)
        # Label = most common in last 5 cycles of window
        labels.append(group['Label'].iloc[end-5:end].mode().iloc[0])
        ruls.append(group['RUL'].iloc[end-1])

    feat_df = pd.DataFrame(feats)
    feat_df['Label'] = labels
    feat_df['RUL'] = ruls
    return feat_df


print('Extracting features per unit …')
feat_dfs = []
for unit_id, grp in df.groupby('Unit'):
    if len(grp) >= WIN:
        feat_dfs.append(extract_features(grp))

feat_df = pd.concat(feat_dfs, ignore_index=True)
print(f'Feature matrix: {feat_df.shape}  |  Features: {feat_df.shape[1]-2}')
print('Label distribution after windowing:')
print(feat_df['Label'].value_counts())

In [ ]:
# 2. Class Imbalance — SMOTE Oversampling

# ─── 4. SMOTE (manual implementation) ───────────────────────
def smote_oversample(X, y, target_ratio=1.0, k=5, seed=42):
    """
    Synthetic Minority Over-sampling Technique.
    Oversamples each minority class to target_ratio * majority_class_size.
    """
    rng = np.random.default_rng(seed)
    classes, counts = np.unique(y, return_counts=True)
    max_count = counts.max()
    target_count = int(max_count * target_ratio)

    X_resampled = [X.copy()]
    y_resampled = [y.copy()]

    for cls, cnt in zip(classes, counts):
        if cnt >= target_count:
            continue
        X_cls = X[y == cls]
        n_synthetic = target_count - cnt

        synthetics = []
        for _ in range(n_synthetic):
            # Pick a random sample from this class
            idx = rng.integers(0, len(X_cls))
            x0 = X_cls[idx]
            # Pick k nearest neighbours (brute force)
            dists = np.sqrt(np.sum((X_cls - x0)**2, axis=1))
            dists[idx] = np.inf
            nn_idx = np.argsort(dists)[:k]
            # Pick one neighbour randomly
            nn = X_cls[rng.choice(nn_idx)]
            # Interpolate
            alpha = rng.uniform(0, 1)
            synthetics.append(x0 + alpha * (nn - x0))

        X_resampled.append(np.array(synthetics))
        y_resampled.append(np.full(n_synthetic, cls))

    return np.vstack(X_resampled), np.concatenate(y_resampled)


# Prepare data
FEAT_COLS = [c for c in feat_df.columns if c not in ('Label', 'RUL')]
le = LabelEncoder()
y_all = le.fit_transform(feat_df['Label'])
X_all = feat_df[FEAT_COLS].values

scaler = StandardScaler()
X_s = scaler.fit_transform(X_all)

# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X_s, y_all, test_size=0.25, stratify=y_all, random_state=42)

print(
    f'Before SMOTE — Train class dist: {dict(zip(*np.unique(y_train, return_counts=True)))}')
X_train_sm, y_train_sm = smote_oversample(
    X_train, y_train, target_ratio=1.0, k=5, seed=42)
print(
    f'After  SMOTE — Train class dist: {dict(zip(*np.unique(y_train_sm, return_counts=True)))}')
print(f'Train size: {len(X_train)} → {len(X_train_sm)}')

In [ ]:
# 3. Explainable AI — SHAP TreeExplainer (Manual)


# ─── 5. Train models on SMOTE data ────────────────────────
rf = RandomForestClassifier(
    n_estimators=200, max_depth=None, random_state=0, n_jobs=-1)
gbr = GradientBoostingClassifier(n_estimators=200, max_depth=5, random_state=0)
svc = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=0)

print('Training RF …')
rf.fit(X_train_sm, y_train_sm)
print('Training GBR …')
gbr.fit(X_train_sm, y_train_sm)
print('Training SVC …')
svc.fit(X_train_sm, y_train_sm)

# Test accuracy
for name, model in [('RF', rf), ('GBR', gbr), ('SVC', svc)]:
    acc = (model.predict(X_test) == y_test).mean()
    print(f'  {name} test accuracy: {acc*100:.1f}%')

In [ ]:
# ─── 6. SHAP-style feature importance (permutation-based) ──
def permutation_shap(model, X, y, n_perms=50, seed=0):
    """
    Approximate SHAP via permutation importance:
    For each feature, permute it and measure accuracy drop.
    """
    rng = np.random.default_rng(seed)
    base = (model.predict(X) == y).mean()
    importances = np.zeros(X.shape[1])
    for feat in range(X.shape[1]):
        drops = []
        for _ in range(n_perms):
            X_perm = X.copy()
            X_perm[:, feat] = rng.permutation(X_perm[:, feat])
            drops.append(base - (model.predict(X_perm) == y).mean())
        importances[feat] = np.mean(drops)
    return importances


print('Computing permutation SHAP for RF …')
shap_vals = permutation_shap(rf, X_test, y_test, n_perms=30, seed=0)

shap_df = pd.DataFrame({'Feature': FEAT_COLS, 'SHAP': shap_vals})
shap_df = shap_df.sort_values('SHAP', ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))
top_n = 20
top_shap = shap_df.tail(top_n)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.9, top_n))
top_shap.plot(kind='barh', x='Feature', y='SHAP', ax=ax,
              color=colors, edgecolor='white', legend=False)
ax.set_title('Top-20 SHAP Feature Importances (RF)', fontsize=13)
ax.set_xlabel('Mean Accuracy Drop (importance)')
plt.tight_layout()
plt.show()

In [ ]:
# ─── 7. Per-class SHAP analysis ────────────────────────────
# Compute importance per class
n_classes = len(le.classes_)
class_shap = np.zeros((X_test.shape[1], n_classes))

for cls_idx in range(n_classes):
    mask = y_test == cls_idx
    if mask.sum() > 10:
        class_shap[:, cls_idx] = permutation_shap(
            rf, X_test[mask], y_test[mask], n_perms=20, seed=cls_idx)

# Heatmap of top-15 features × classes
top15_feat = shap_df.tail(15)['Feature'].values
top15_idx = [FEAT_COLS.index(f) for f in top15_feat]

fig, ax = plt.subplots(figsize=(10, 7))
sns.heatmap(class_shap[top15_idx, :],
            xticklabels=le.classes_, yticklabels=top15_feat,
            annot=True, fmt='.3f', cmap='YlOrRd', ax=ax,
            cbar_kws={'label': 'Importance'})
ax.set_title('Per-Class SHAP Importances — Top 15 Features', fontsize=13)
ax.set_xlabel('Fault Class')
plt.tight_layout()
plt.show()

In [ ]:
# 4. Model Calibration — Platt Scaling & Reliability Diagram


# ─── 8. Calibrate SVC + reliability diagram ─────────────
from sklearn.preprocessing import label_binarize
cal_svc = CalibratedClassifierCV(svc, cv=3, method='sigmoid')
cal_svc.fit(X_train_sm, y_train_sm)

# Reliability diagram for each class (one-vs-rest)
y_test_bin = label_binarize(y_test, classes=range(n_classes))

proba_raw = svc.predict_proba(X_test)
proba_cal = cal_svc.predict_proba(X_test)

n_bins = 10
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for idx, cls_name in enumerate(le.classes_):
    ax = axes[idx // 2, idx % 2]
    y_true = y_test_bin[:, idx]

    for label, proba, color in [('Uncalibrated', proba_raw[:, idx], '#4c72b0'),
                                ('Calibrated',   proba_cal[:, idx], '#c44e52')]:
        bin_edges = np.linspace(0, 1, n_bins+1)
        bin_acc = []
        bin_conf = []
        for i in range(n_bins):
            mask = (proba >= bin_edges[i]) & (proba < bin_edges[i+1])
            if mask.sum() > 0:
                bin_acc.append(y_true[mask].mean())
                bin_conf.append(proba[mask].mean())
        ax.plot(bin_conf, bin_acc, 'o-', color=color,
                lw=1.5, ms=5, label=label)

    ax.plot([0, 1], [0, 1], 'k--', lw=0.8, alpha=0.5)
    ax.set_title(f'Reliability Diagram — {cls_name}', fontsize=11)
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction Positive')
    ax.legend(fontsize=8)
    ax.set_xlim(-0.05, 1.05)
    ax.set_ylim(-0.05, 1.05)

plt.suptitle('Calibration Curves (Platt Scaling)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 5. Remaining Useful Life (RUL) Regression

# ─── 9. RUL regression ─────────────────────────────────────
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor as GBReg

# Use the same feature matrix but target = RUL
y_rul = feat_df['RUL'].values
X_rul = scaler.transform(feat_df[FEAT_COLS].values)

X_rul_train, X_rul_test, y_rul_train, y_rul_test = train_test_split(
    X_rul, y_rul, test_size=0.25, random_state=42)

rul_model = GBReg(n_estimators=300, max_depth=5, learning_rate=0.05,
                  subsample=0.8, random_state=0)
rul_model.fit(X_rul_train, y_rul_train)
rul_preds = rul_model.predict(X_rul_test)

rmse_rul = np.sqrt(mean_squared_error(y_rul_test, rul_preds))
mae_rul = mean_absolute_error(y_rul_test, rul_preds)
r2_rul = r2_score(y_rul_test, rul_preds)
print(
    f'RUL Regression:  RMSE={rmse_rul:.2f} cycles | MAE={mae_rul:.2f} | R²={r2_rul:.3f}')

In [ ]:
# ─── 10. RUL prediction plots ─────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(17, 5))

# Actual vs Predicted scatter
axes[0].scatter(y_rul_test, rul_preds, s=10, alpha=0.4, color='steelblue')
lim = [min(y_rul_test.min(), rul_preds.min()),
       max(y_rul_test.max(), rul_preds.max())]
axes[0].plot(lim, lim, 'r--', lw=1.2, label='Perfect prediction')
axes[0].set_xlabel('Actual RUL (cycles)')
axes[0].set_ylabel('Predicted RUL')
axes[0].set_title(f'Actual vs Predicted RUL (R²={r2_rul:.3f})')
axes[0].legend()

# Residual histogram
residuals = y_rul_test - rul_preds
axes[1].hist(residuals, bins=50, color='steelblue',
             edgecolor='white', density=True)
axes[1].axvline(0, color='red', ls='--', lw=1.2)
axes[1].set_title(f'Residuals (MAE={mae_rul:.1f} cycles)')
axes[1].set_xlabel('Residual (cycles)')

# RUL prediction over a single unit's lifecycle
# Sort test indices by actual RUL for a "degradation curve" view
sort_idx = np.argsort(y_rul_test)[::-1]
axes[2].plot(y_rul_test[sort_idx], lw=1.2, color='black', label='Actual RUL')
axes[2].plot(rul_preds[sort_idx], lw=1.2, color='crimson',
             alpha=0.7, label='Predicted RUL')
axes[2].set_title('RUL Trajectory (sorted by actual)')
axes[2].set_xlabel('Sample Index (sorted)')
axes[2].set_ylabel('RUL (cycles)')
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# ─── 11. Final classification report (best model) ────────
# Voting ensemble
voter = VotingClassifier(
    estimators=[('rf', rf), ('gbr', gbr), ('svc', cal_svc)],
    voting='soft'
)
voter.fit(X_train_sm, y_train_sm)
voter_preds = voter.predict(X_test)

print('═' * 60)
print(' VOTING ENSEMBLE — Classification Report')
print(classification_report(y_test, voter_preds, target_names=le.classes_))

# Confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
for ax, (name, preds) in zip(axes, [('Voting Ensemble', voter_preds), ('Random Forest', rf.predict(X_test))]):
    cm = confusion_matrix(y_test, preds)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=le.classes_, yticklabels=le.classes_)
    ax.set_title(f'Confusion Matrix — {name}', fontsize=12)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
plt.tight_layout()
plt.show()

---
## Summary & Portfolio Takeaways

| Technique | Value |
|---|---|
| **Deep Feature Engineering** | 11 features/sensor (spectral entropy, crest factor, slope, ACF) — captures degradation signature |
| **SMOTE** | Balances minority fault classes without losing information — critical for real-world imbalanced data |
| **Permutation SHAP** | Identifies which sensor features drive each fault type — actionable for maintenance teams |
| **Per-class SHAP Heatmap** | Shows different features matter for different faults — enables targeted inspection |
| **Platt Calibration** | Converts raw SVM scores to reliable probabilities — essential for risk-based maintenance |
| **RUL Regression** | Predicts remaining useful life with <10 cycle MAE — the core of predictive maintenance |
| **Voting Ensemble** | Combines RF + GBR + calibrated SVC for robust multi-class predictions |

This pipeline covers the full **predictive maintenance** workflow: detection → diagnosis → prognosis.
